In [ ]:
%pip install matplotlib pandas

In [ ]:
import glob
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
csv_filename = sorted(glob.glob("./result-*.csv"))[-1]
print(csv_filename)

df = pd.read_csv(csv_filename)
df.head(10)

In [ ]:
# Add integer version of transfer_size column
def size_str_to_int(s):
    if s.endswith("K"):
        i = int(s[:-1])
    elif s.endswith("M"):
        i = int(s[:-1]) * 1024
    elif s.endswith("G"):
        i = int(s[:-1]) * 1024 * 1024
    elif s.endswith("T"):
        i = int(s[:-1]) * 1024 * 1024 * 1024
    else:
        assert False
    return i

df["transfer_size_int"] = df["transfer_size"].map(size_str_to_int)

df.head()

In [ ]:
df.dtypes

In [ ]:
# test-1: granularity x 3 variations, number of processes x 9 variations

def plot_graph( title, condition, values ):

    df2 = df[condition]
    df3 = df2.pivot(index="num_processes", columns=["filesystem_type"], values=values)
    df3.plot(title=title)

plot_graph( "Read 4K", df["transfer_size"]=="4K", ["read_bw_mean"] )
plot_graph( "Write 4K", df["transfer_size"]=="4K", ["write_bw_mean"] )

plot_graph( "Read 1M", df["transfer_size"]=="1M", ["read_bw_mean"] )
plot_graph( "Write 1M", df["transfer_size"]=="1M", ["write_bw_mean"] )

plot_graph( "Read 256M", df["transfer_size"]=="256M", ["read_bw_mean"] )
plot_graph( "Write 256M", df["transfer_size"]=="256M", ["write_bw_mean"] )

In [ ]:
# test-2: number of processes = 32, granularity x 17, variations

def plot_graph( title, condition, values ):

    df2 = df[condition]
    df3 = df2.pivot(index="transfer_size_int", columns=["filesystem_type"], values=values)
    df3.plot(title=title)

plot_graph( "Read 32 processes", df["num_processes"]==32, ["read_bw_mean"] )
plot_graph( "Write 32 processes", df["num_processes"]==32, ["write_bw_mean"] )
